# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [3]:
from pymongo import MongoClient
import pandas as pd
# import time

In [4]:
client = MongoClient("localhost:27017")
db = client["Ironhack"]
c = db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [193]:
list(c.find())[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [192]:
list(c.find())[5]


{'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
 'name': 'Omnidrive',
 'permalink': 'omnidrive',
 'crunchbase_url': 'http://www.crunchbase.com/company/omnidrive',
 'homepage_url': 'http://www.omnidrive.com',
 'blog_url': 'http://www.omnidrive.com/blog',
 'blog_feed_url': 'http://feeds.feedburner.com/omnidrive',
 'twitter_username': 'Nomadesk',
 'category_code': 'network_hosting',
 'number_of_employees': None,
 'founded_year': 2005,
 'founded_month': 11,
 'founded_day': 1,
 'deadpooled_year': 2008,
 'deadpooled_month': 9,
 'deadpooled_day': 15,
 'deadpooled_url': '',
 'tag_list': 'storage, sharing, edit, online',
 'alias_list': None,
 'email_address': 'info@omnidrive.com',
 'phone_number': '660-675-5052',
 'description': None,
 'created_at': 'Sun May 27 03:25:32 UTC 2007',
 'updated_at': 'Tue Jul 02 22:48:04 UTC 2013',
 'overview': '<p>Currently in public beta, Omnidrive makes it easy to access, edit and share your files from any computer with a web browser. The site was founded in late 2

In [67]:
filter_Babel = {"name": "Babelgum"}
projection_Babel = {"name":1, "_id": 0}
list(c.find(filter_Babel, projection_Babel))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [109]:
filter_employees = {"number_of_employees": {"$gte": 5000}}
projection_employees = {"name":1, "number_of_employees":1, "_id":0} # For legibility and to check the result
list(c.find(filter_employees, projection_employees).sort("number_of_employees", 1).limit(20))[:10] # Ordered ascendently

[{'name': 'JDS Uniphase Corporation', 'number_of_employees': 5000},
 {'name': 'PCH International', 'number_of_employees': 5000},
 {'name': 'Vcare Call Center', 'number_of_employees': 5000},
 {'name': 'United Internet', 'number_of_employees': 5000},
 {'name': 'McAfee', 'number_of_employees': 5000},
 {'name': 'Spotify', 'number_of_employees': 5000},
 {'name': 'Air Astana Airlines', 'number_of_employees': 5000},
 {'name': 'Nintendo', 'number_of_employees': 5080},
 {'name': 'Hexaware Technologies', 'number_of_employees': 5200},
 {'name': 'Facebook', 'number_of_employees': 5299}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [66]:
filter_found = {"founded_year": {"$gte": 2000, "$lte": 2005}}
projection_founded = {"name":1, "founded_year":1, "_id":0}
list(c.find(filter_found, projection_founded))[:5]

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [69]:
condition_val1 = {"ipo.valuation_amount":{"$gte": 100000000}}
condition_val2 = {"founded_year": {"$lt": 2010}}
projection_valuation = {"name": 1, "ipo": 1, "_id": 0}
list(c.find({"$and": [condition_val1, condition_val2]}, projection_valuation))[:5]

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [123]:
cond_empfo1 = {"number_of_employees":{"$lt": 1000}}
cond_empfo2 = {"founded_year": {"$lt": 2005}}
projection_empfo = {"name":1, "number_of_employees":1, "founded_year": 1, "_id":0} # For legibility and to check the result
list(c.find({"$and": [cond_empfo1, cond_empfo2]}, projection_empfo).sort("number_of_employees", 1).limit(10))

[{'name': 'Skype', 'number_of_employees': 0, 'founded_year': 2003},
 {'name': 'Simpy', 'number_of_employees': 0, 'founded_year': 2004},
 {'name': 'Eurekster', 'number_of_employees': 0, 'founded_year': 2004},
 {'name': 'Ticketmaster', 'number_of_employees': 0, 'founded_year': 1976},
 {'name': 'Fox Interactive Media',
  'number_of_employees': 0,
  'founded_year': 1979},
 {'name': 'stylediary', 'number_of_employees': 0, 'founded_year': 2004},
 {'name': 'Monster', 'number_of_employees': 0, 'founded_year': 1994},
 {'name': 'EditGrid', 'number_of_employees': 0, 'founded_year': 2003},
 {'name': 'Compete', 'number_of_employees': 0, 'founded_year': 2000},
 {'name': 'MindTouch', 'number_of_employees': 0, 'founded_year': 2004}]

### 6. All the companies that don't include the `partners` field.

In [102]:
list(c.find({"partners":{"$exists":False}}))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [125]:
projection_null = {"name":1, "category_code":1, "_id":0} # For legibility and to check the result
list(c.find({"category_code":{"$type": "null"}}, projection_null))[:5]

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [128]:
cond_emp1 = {"number_of_employees": {"$gte": 100}}
cond_emp2 = {"number_of_employees": {"$lt": 1000}}
projection_emp = {"name":1, "number_of_employees":1, "_id":0}
list(c.find({"$and": [cond_emp1, cond_emp2]}, projection_emp))[:5]

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [137]:
projection = {"name":1, "ipo.valuation_amount":1, "_id":0} # For legibility and to check the result
list(c.find(projection = projection).sort("ipo.valuation_amount", -1))[:5]

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [142]:
projection = {"name":1, "number_of_employees":1, "_id":0} # For legibility and to check the result
list(c.find(projection = projection).sort("number_of_employees", -1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [144]:
filter_sem = {"founded_month": {"$gt":6}}
projection = {"name":1, "founded_month":1, "_id":0} # For legibility and to check the result
list(c.find(filter_sem, projection).limit(1000))[:5]

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'eBay', 'founded_month': 9}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [145]:
cond_year1 = {"founded_year": {"$lt":2000}}
cond_year2 = {"acquisition.price_amount": {"$gt" : 10000000}}
projection = {"name":1, "founded_year":1, "acquisition.price_amount":1, "_id":0} # For legibility and to check the result
list(c.find({"$and": [cond_year1, cond_year2]}, projection))[:5]

[{'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep',
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar',
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'name': 'PayPal',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish',
  'founded_year': 1999,
  'acquisition': {'price_amount': 300000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [148]:
filter_acq = {"acquisition.acquired_year": {"$gt":2010}}
projection_acq = {"name":1, "acquisition":1, "_id":0}
list(c.find(filter_acq, projection_acq).sort("acquisition.price_amount", 1))[:5]

[{'name': 'Geni',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2012/11/28/all-in-the-family-myheritage-buys-former-yammer-stablemate-geni-com-raises-25m/',
   'source_description': 'MyHeritage acquires Geni and $25M to build family tree of the whole world',
   'acquired_year': 2012,
   'acquired_month': 11,
   'acquired_day': 28,
   'acquiring_company': {'name': 'MyHeritage', 'permalink': 'myheritage'}}},
 {'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'blogT

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [168]:
projection_found = {"name":1, "founded_year":1, "_id":0}
list(c.find(projection = projection_found).sort("founded_year", 1))[:5]

[{'name': 'Flektor', 'founded_year': None},
 {'name': 'Lala', 'founded_year': None},
 {'name': 'SpinVox', 'founded_year': None},
 {'name': 'Gannett', 'founded_year': None},
 {'name': 'Info', 'founded_year': None}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [184]:
filter_week = {"founded_day": {"$lte":7}}
projection = {"name":1, "founded_day":1, "acquisition.price_amount":1, "_id":0} # For legibility and to check the results
list(c.find(filter_week, projection).sort("acquisition.price_amount", -1).limit(10))[:5]

[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000.0}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [183]:
cond_web1 = {"category_code": "web"}
cond_web2 = {"number_of_employees": {"$gt": 4000}}
projection = {"name":1, "category_code":1, "number_of_employees":1, "_id":0} # For legibility and to check the results
list(c.find({"$and":[cond_web1, cond_web2]}, projection).sort("number_of_employees", 1))[:5]

[{'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [182]:
cond_amount1 = {"acquisition.price_amount": {"$gt": 10000000}}
cond_amount2 = {"acquisition.price_currency_code":"EUR"}
projection = {"name":1, "acquisition.price_amount":1, "acquisition.price_currency_code":1, "_id":0} # For legibility and to check the results
list(c.find({"$and":[cond_amount1, cond_amount2]}, projection))[:5]

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [187]:
filter_tri = {"acquisition.acquired_month": {"$lt": 4}}
projection_tri = {"name":1, "acquisition":1, "_id":0}
list(c.find(filter_tri, projection_tri).limit(10))[:5]

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [189]:
cond_2000_1 = {"founded_year": {"$gte": 2000}}
cond_2000_2 = {"founded_year": {"$lte": 2010}}
cond_2000_3 = {"acquisition.acquired_year": {"$gte": 2011}}
projection = {"name":1, "founded_year":1, "acquisition.acquired_year":1, "_id":0} # For legibility and to check the results
list(c.find({"$and":[cond_2000_1, cond_2000_2, cond_2000_3]}, projection))[:5]

[{'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Kyte',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2011}},
 {'name': 'Jingle Networks',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2011}}]

### 20. All the companies that have been 'deadpooled' after the third year.

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```